<a href="https://colab.research.google.com/github/valefe/playground_transformers/blob/main/Playground_Trying_multiple_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture magic
!pip install bpe
!pip install transformers

In [ ]:
from torch import nn, tensor
import torch
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("description_bisac.csv", nrows=5000, names=["id", "mainDescription", "bisacCode"])
# TO-DO: Mask some words

# Tokenizing the data

In [ ]:
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()

In [ ]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(["./description_bisac.csv"], trainer=trainer)

*"Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."*

In [ ]:
o_alquimista = "Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."
masked_o_alquimista = "Quando você quer alguma coisa, todo o [MASK] conspira para que você realize seu [MASK]. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."
encoded = tokenizer.encode(o_alquimista)
encoded.ids[:20]


[2800, 443, 1336, 122, 62, 140, 105, 1993, 11, 860]

# Creating a Dataset 

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MetabooksDataset(Dataset):
    """Metabooks dataset."""

    def __init__(self, csv_file, tokenizer):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            tokenizer (huggingface Tokenizer): Will encode str sentences
        """
        self.genres_df = pd.read_csv("description_bisac.csv", names=["id", "mainDescription", "bisacCode"])
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        encoded = self.tokenizer.encode(self.genres_df["mainDescription"][idx])
        encoded_size = len(encoded.ids)
        # TO-DO: Encode as one hot
        label = self.genres_df["bisacCode"].astype("category").cat.codes[idx]
        return {"text": tensor(encoded.ids, dtype=torch.int64).view(1, encoded_size), "label": tensor(label, dtype=torch.int64)}

In [ ]:
mb_dataset = MetabooksDataset('description_bisac.csv', tokenizer)

In [ ]:
mb_dataset[0]

# Fine Tuning

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
from tqdm import tqdm

for i in tqdm(range(20)):
  data = mb_dataset[0]
  outputs = model("Eu sou una menina", labels=data["label"])
  loss = outputs.loss
  loss.backward()
  optimizer.step()

  0%|          | 0/20 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import BertForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework="pt")

# Results

In [ ]:
o_alquimista = "Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."
encoded = tokenizer.encode(o_alquimista)
ids = encoded.ids
model(tensor(ids, dtype=torch.int64).view(1, len(ids)))